# Đồ án cuối kỳ
## Nhóm 07

### Thành viên
1. Lưu Hữu Tình - 21424056
2. Đặng Hồ Hoàng Duy - 21424073
3. Huỳnh Văn Thái - 21424088

<h1>
<font >Chủ đề: THU THẬP DỮ LIỆU THÔNG TIN CÁC ĐỘI BÓNG</font>
</h1>

<h1>
<font color="blue">PHẦN 1: QUY TRÌNH KHOA HỌC DỮ LIỆU</font>
</h1>

<h2>
<font color="blue">A. Thu thập dữ liệu</font>
</h2>

<h4>
<font >1A/ Cài đặt các thư viện, gói cho việc collect data</font>
</h4>

In [51]:
!pip install scrapy
!pip install spider3

In [ ]:
import json
import re
import pandas as pd
import scrapy
from pandas.testing import assert_frame_equal
import datetime

<h4>
<font >2A/ Start project, thiết lập đường dẫn</font>
</h4>

In [9]:
!scrapy startproject fifa_crawler

Error: Module 'fifa_crawler' already exists


In [16]:
cd C:/Users/ASUS/fifa_crawler/fifa_crawler

C:\Users\ASUS\fifa_crawler\fifa_crawler


In [20]:
!ls
!touch spiders/collect_teams_urls.py

'ls' is not recognized as an internal or external command,
operable program or batch file.
'touch' is not recognized as an internal or external command,
operable program or batch file.


<h4>
<font >3A/ Tạo file collect_teams_urls.py</font>
</h4>

1. File sẽ collect data từ trang https://sofifa.com/teams?col=oa&sort=desc&offset=0
2. Cấu trúc trả về là json, danh sách các url của mỗi đội bóng
```json
[
{"team_url": "/team/10"},
{"team_url": "/team/21"},
...
]
```

In [54]:
# File: \fifa_crawler\fifa_crawler\spiders\collect_teams_urls.py
import scrapy

class collect_team_url(scrapy.Spider):
  name='teams_urls' 
  
  def start_requests(self):
    urls = ['https://sofifa.com/teams?col=oa&sort=desc&offset=0']
    
    # YOUR CODE HERE
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

    for i in range(1,13):
        url = f'https://sofifa.com/teams?col=oa&sort=desc&offset={i*60}' 
        urls.append(url)

    for url in urls:
      yield scrapy.Request(url=url, headers=headers, callback=self.parse)

  def parse(self, response):
    team_urls = response.css('td.col-name-wide a::attr(href)').extract()
    team_ids = []
    for url in team_urls:
      if url.split('/')[1] == "team":
        team_id = url.split('/')[2]
        team_url = f"/team/{team_id}"
        team_item = {"team_url": team_url}
        yield team_item

<h4>
<font >4A/ Start collect data từ file collect_teams_urls.py</font>
</h4>

- Data collect được lưu vào dataset/teams_urls.json

In [22]:
!scrapy crawl teams_urls -o dataset/teams_urls.json

2023-04-15 18:00:12 [scrapy.utils.log] INFO: Scrapy 2.8.0 started (bot: fifa_crawler)
2023-04-15 18:00:12 [scrapy.utils.log] INFO: Versions: lxml 4.5.2.0, libxml2 2.9.10, cssselect 1.2.0, parsel 1.7.0, w3lib 2.1.1, Twisted 22.10.0, Python 3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 23.0.0 (OpenSSL 3.0.8 7 Feb 2023), cryptography 39.0.2, Platform Windows-10-10.0.22000-SP0
2023-04-15 18:00:12 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'fifa_crawler',
 'FEED_EXPORT_ENCODING': 'utf-8',
 'NEWSPIDER_MODULE': 'fifa_crawler.spiders',
 'REQUEST_FINGERPRINTER_IMPLEMENTATION': '2.7',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['fifa_crawler.spiders'],
 'TWISTED_REACTOR': 'twisted.internet.asyncioreactor.AsyncioSelectorReactor'}
2023-04-15 18:00:12 [asyncio] DEBUG: Using selector: SelectSelector
2023-04-15 18:00:12 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2023-04-15 18:00:12 [scrapy.utils.log] D

In [62]:
df_test = pd.read_json('dataset/teams_urls.json', encoding='utf-8-sig')
df_test.iloc[0]

ValueError: Trailing data

<h4>
<font >5A/ Tạo file collect_teams_info.py</font>
</h4>

1. Thu thập thông tin chi tiết của mỗi đội bóng
2. Trả về data dạng json
```json
[
{
    "Name": "Chelsea", 
    "League": "[England] Premier League", 
    "Overall": "83", 
    "Attack": "83", 
    "Midfield": "81", 
    "Defence": "82", 
    "Home stadium": "Stamford Bridge", 
    "Rival team": "Arsenal", 
    "International prestige": "8", 
    "Domestic prestige": "9", 
    "Club worth": "\n", 
    "Starting XI average age": "24.55", 
    "Whole team average age": "24.39", 
    "Captain": "R. James", 
    "Short free kick": "R. James", 
    "Long free kick": "E. Fernández", 
    "Left short free kick": "João Félix", 
    "Right short free kick": "R. James", 
    "Penalties": "K. Havertz", 
    "Left corner": "B. Chilwell", 
    "Right corner": "R. James"
},
    ...
]
```

In [56]:
# \fifa_crawler\fifa_crawler\spiders\collect_teams_info.py
import scrapy
import json
import re
import datetime

class collect_team_info(scrapy.Spider):
  name='teams_info'
  
  def __init__(self):
    try:
      with open('dataset/teams_urls.json') as f:
        self.teams = json.load(f)
      self.team_count = 1
    except IOError:
      print("File not found")

  def start_requests(self):
    # YOUR CODE HERE
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    
    for team in self.teams:
      url = 'https://sofifa.com' + team['team_url'] + '?units=mks'
      yield scrapy.Request(url=url, headers=headers, callback=self.parse)
  
  def parse(self, response):
      # YOUR CODE HERE
      team_info = {}

      team_info["Name"] = response.css('div.info h1::text').get();

      team_info["League"] = response.css('div.meta.ellipsis a::text').get()

      ratings = response.css('section.card .bp3-tag.p::text').getall()

      team_info["Overall"] = ratings[0]

      team_info["Attack"] = ratings[1]
      
      team_info["Midfield"] = ratings[2]
      
      team_info["Defence"] = ratings[3]

      team_info["Home stadium"] = response.css('ul.pl li.ellipsis:contains("Home stadium")::text').get()
      
      team_info["Rival team"] = response.css('ul.pl li:contains("Rival team") a::text').get()

      team_info["International prestige"] = response.css('ul.pl li:contains("International prestige") span::text').get()

      team_info["Domestic prestige"] = response.css('ul.pl li:contains("Domestic prestige") span::text').get()

      team_info["Club worth"] = response.css('ul.pl li:contains("Club worth")::text').get()

      team_info["Starting XI average age"] = response.css('ul.pl li:contains("Starting XI average age")::text').get()

      team_info["Whole team average age"] = response.css('ul.pl li:contains("Whole team average age")::text').get()

      team_info["Captain"] = response.css('ul.pl li:contains("Captain") a::text').get()

      team_info["Short free kick"] = response.css('ul.pl li:contains("Short free kick") a::text').get()

      team_info["Long free kick"] = response.css('ul.pl li:contains("Long free kick") a::text').get()

      team_info["Left short free kick"] = response.css('ul.pl li:contains("Left short free kick") a::text').get()
      
      team_info["Right short free kick"] = response.css('ul.pl li:contains("Right short free kick") a::text').get()
      
      team_info["Penalties"] = response.css('ul.pl li:contains("Penalties") a::text').get()
      
      team_info["Left corner"] = response.css('ul.pl li:contains("Left corner") a::text').get()
      
      team_info["Right corner"] = response.css('ul.pl li:contains("Right corner") a::text').get()

      yield team_info

In [ ]:
<h4>
<font >6A/ Start collect data từ file collect_teams_info.py</font>
</h4>

- Data collect được lưu vào dataset/teams_info.json

In [50]:
#!rm /content/fifa_crawler/fifa_crawler/dataset/teams_info.json
!scrapy crawl teams_info -o dataset/teams_info.json


2023-04-15 18:16:51 [scrapy.spidermiddlewares.httperror] INFO: Ignoring response <429 https://sofifa.com/team/1800?units=mks>: HTTP status code is not handled or not allowed
2023-04-15 18:16:51 [scrapy.spidermiddlewares.httperror] INFO: Ignoring response <429 https://sofifa.com/team/112134?units=mks>: HTTP status code is not handled or not allowed
2023-04-15 18:16:51 [scrapy.spidermiddlewares.httperror] INFO: Ignoring response <429 https://sofifa.com/team/263?units=mks>: HTTP status code is not handled or not allowed
2023-04-15 18:16:51 [scrapy.downloadermiddlewares.retry] DEBUG: Retrying <GET https://sofifa.com/team/111144?units=mks> (failed 1 times): 429 Unknown Status
2023-04-15 18:16:51 [scrapy.downloadermiddlewares.retry] ERROR: Gave up retrying <GET https://sofifa.com/team/1801?units=mks> (failed 3 times): 429 Unknown Status
2023-04-15 18:16:51 [scrapy.core.engine] DEBUG: Crawled (429) <GET https://sofifa.com/team/1801?units=mks> (referer: None)
2023-04-15 18:16:51 [scrapy.downl

<h4>
<font >6A/ Đọc thông tin collect được lưu vào biến df_teams</font>
</h4>

In [70]:
df_teams = pd.read_json('dataset/teams_info.json', encoding='utf-8-sig',lines=True)
df_teams.iloc[0]

ValueError: Expected object or value

<h2>
<font color="blue">B. Khám phá dữ liệu</font>
</h2>

<h2>
<font color="blue">C Đặt các câu hỏi có ý nghĩa cần trả lời</font>
</h2>

<h1>
<font color="blue">PHẦN 2: MÔ HÌNH HÓA DỮ LIỆU</font>
</h1>

<h2>
<font color="blue">A. Mô hình hóa dữ liệu và đánh giá mô hình
</font>
</h2>

<h2>
<font color="blue">B. Đánh giá mô hình</font>
</h2>